참고: https://tutorials.pytorch.kr/beginner/pytorch_with_examples.html
## PyTorch: Tensors
- Tensor: 개념적으로 numpy 배열과 동일
    - N차원 배열
    - GPU 활용해 수치 연산 가속화 가능

In [13]:
import torch

dtype = torch.float
device = torch.device("cpu")

# N: batch size / H: 은닉층 차원
# D_in: 입력 차원 / D_out: 출력 차원
N, D_in, H, D_out = 64, 1000, 100, 10

# 랜덤으로 입력과 출력 데이터 생성
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 랜덤으로 가중치 초기화
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6

for t in range(500):
    # forward: prediction 계산
    h = x.mm(w1)
    h_relu = h.clamp(min = 0)
    y_pred = h_relu.mm(w2)
    
    # loss 계산하고 출력
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)
    
    # backpropagation: loss에 따른 w1, w2의 변화도 계산
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)
    
    # gradient descent
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

99 285.5345458984375
199 1.9994640350341797
299 0.03371996805071831
399 0.0009843886364251375
499 0.00013144993863534182


## Tensor와 autograd
- 이렇게 작은 신경망에서 역전파를 직접 구현하는 것은 큰 일이 아니지만, 대규모의 복잡한 신경망에서는 힘든 일이다.
- 대신 autograd를 사용하면 신경망에서 역전파 단계의 연산을 자동화할 수 있다.
- Autograd를 사용할 때, 신경망의 순전파 단계에서는 연산 그래프를 정의하게 된다.
    - 이 그래프의 노드, 텐서, 엣지는 입력 Tensor로부터 출력 Tensor를 만들어내는 함수가 된다.
    - 이 그래프를 통해 역전파를 하게 되면 변화도도 쉽게 계산할 수 있다.
- 각 Tensor는 연산 그래프에서 노드로 표현된다.
    - 만약 x가 x.requires_trad=True인 Tensor라면 x.grad는 어떤 스칼라값에 대한 x의 변화도를 갖는 또 다른 Tensor가 된다.

In [14]:
import torch

dtype = torch.float
device = torch.device("cpu")

N, D_in, h, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

w1 = torch.randn(D_in, h, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(h, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6

for t in range(500):
    # clamp: 괄호 안의 값의 범주에 해당하도록 값을 변경
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    loss = (y_pred - y).pow(2).sum()
    
    if t % 100 == 99:
        print(t, loss.item())
    
    # autograd를 사용해 역전파 단계 계산: requires_grad=True를 갖는 모든 Tensor에 대한 손실 변화도 계산
    # 이후 w1.grad와 w2.grad는 w1와 w2 각각에 대한 손실의 변화도를 갖는 Tensor가 된다.
    loss.backward()
    
    # gradient descent 사용해 가중치 수동으로 갱신
    # torch.no_grad()로 감싸는 이유: 가중치들이 requires_grad=True 이지만 autograd에서는 이를 추적할 필요 없기 때문!
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # 가중치 갱신 후 수동으로 변화도를 0으로 만든다.
        w1.grad.zero_()
        w2.grad.zero_()
    

99 270.93780517578125
199 1.073225498199463
299 0.007113119587302208
399 0.00018978930893354118
499 3.812230352195911e-05


## 새 autograd 함수 정의하기
- autograd의 기본 연산자는 실제로 Tensor를 조작하는 2개의 함수
    - forward: 입력 Tensor로부터 출력 Tensor를 계산
    - backward: 어떤 스칼라 값에 대한 출력 Tensor의 변화도를 전달 받고, 동일한 스칼라 값에 대한 입력 Tensor의 변화도를 계산한다.
- torch.autograd.Function의 서브클래스를 정의하고 forward와 backward 함수를 구현함으로써 사용자 정의 autograd 연산자를 손쉽게 정의할 수 있다.
    - 그 후 객체를 생성하고 이를 함수처럼 호출해 입력 데이터를 갖는 Tensor를 전달하는 식으로 새로운 autograd 연산자 사용할 수 있다.
    
- ReLU를 이용해 비선형적으로 동작하는 사용자 정의 autograd 함수를 정의 + 2계층 신경망에 적용

In [23]:
import torch

class MyReLU(torch.autograd.Function):
    """
    torch.autograd.Function을 상속받아 사용자 정의 autograd Function 구현하고
    Tensor 연산을 하는 forward와 backward를 구현
    """

    @staticmethod
    def forward(ctx, input):
        """
        입력을 갖는 Tensor를 받아 출력을 갖는 Tensor를 반환
        ctx는 context object로 역전파 연산을 위한 정보 저장에 사용
        ctx.save_for_backward를 사용해 역전파 단계에서 사용할
        어떠한 객체도 저장(cache)해 둘 수 있다.
        """
        
        ctx.save_for_backward(input)
        return input.clamp(min=0)
    
    @staticmethod
    def backward(ctx, grad_output):
        """
        출력에 대한 손실의 변화도를 갖는 Tensor를 받고, 입력에 대한
        손실의 변화도를 계산
        """
        
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        
        return grad_input

In [24]:
dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # GPU에서 실행하려면 이 주석을 제거하세요.

# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 가중치를 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6

for t in range(500):
    # 사용자 정의 Function을 적용하기 위해 Function.apply 메소드를 사용합니다.
    # 여기에 'relu'라는 이름을 붙였습니다.
    relu = MyReLU.apply

    # 순전파 단계: Tensor 연산을 사용하여 예상되는 y 값을 계산합니다;
    # 사용자 정의 autograd 연산을 사용하여 ReLU를 계산합니다.
    y_pred = relu(x.mm(w1)).mm(w2)

    # 손실을 계산하고 출력합니다.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # autograde를 사용하여 역전파 단계를 계산합니다.
    loss.backward()

    # 경사하강법(gradient descent)을 사용하여 가중치를 갱신합니다.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # 가중치 갱신 후에는 수동으로 변화도를 0으로 만듭니다.
        w1.grad.zero_()
        w2.grad.zero_()

99 1488.951904296875
199 17.90707778930664
299 0.27688342332839966
399 0.005417732056230307
499 0.0003151114797219634


# nn 모듈
## PyTorch:nn
- 신경망 계층들과 거의 동일한 Module의 집합을 정의한다.
    - 입력 Tensor를 받고 출력 Tensor를 계산하는 한편, 학습 가능한 매개변수를 갖는 Tensor 같은 inner state를 갖는다.
    - 손실 함수들도 정의

In [25]:
import torch

N, D_in, h, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# nn 패키지를 사용해 모델을 sequence of layers로 정의한다.
# nn.Sequential은 다른 모듈을 포함하는 모듈로 그 모듈들을 순차적으로 적용해 출력을 생성
# 각각의 Linear Module은 선형 함수를 사용해 입력으로부터 출력을 계산하고, 내부 Tensor에 가중치와 편향 저장
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out)
)

# 또한 nn 패키지에는 널리 사용하는 손실 함수들에 대한 정의도 포함하고 있음
# 여기에서는 평균 제곱 오차(MSE)를 손실 함수로 사용해보자
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4

for t in range(500):
    # forward: 모델에 x를 전달해 예상되는 y값을 계산
    # Module 객체는 __call__ 연산자를 덮어써(overrride) 함수처럼 호출할 수 있게 한다.
    # 이렇게 함으로써 입력 데이터의 Tensor를 모듈에 전달해 출력 데이터의 Tensor를 생성
    y_pred = model(x)
    
    # 손실을 계산하고 출력한다.
    # 예측한 y와 정답인 y를 갖는 Tensor들을 전달하고, 손실 함수는 손실 값을 갖는 Tensor를 반환
    loss = loss_fn(y_pred, y)
    
    if t % 100 == 99:
        print(t, loss.item())
        
    # backpropagation 전 변화도 0으로 만들기
    model.zero_grad()
    
    # backpropagation: 모델의 학습 가능한 모든 매개변수에 대해 gradient를 계산
    # 내부적으로 각 모듈의 매개변수는 requires_grad=True일 때 Tensor 내 저장되므로
    # 이 호출을 통해 모든 모델의 모든 학습 가능한 매개변수의 변화도를 계산하게 된다.
    loss.backward()
    
    # gradient descent를 사용해 가중치를 갱신한다.
    # 각 매개변수는 Tensor이므로 이전에 했던 것과 같이 gradient에  접근할 수 있다.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

99 2.4324629306793213
199 0.03542010858654976
299 0.001189535134471953
399 6.883054447825998e-05
499 5.681963102688314e-06


## Pytorch: optim
- 지금까지는 학습 가능한 매개변수를 갖는 Tensor를 직접 조작하며 모델의 가중치를 직접 갱신
    - SGD와 같은 간단한 최적화 알고리즘에서는 크게 부담이 되지 않지만, 실제로 신경망을 학습할 때에는 AdaGrad, RMSProp, Adam과 같은 좀 더 정교한 optimizer를 사용하곤 한다.
- optim 패키지: 최적화 알고리즘에 대한 아이디어 추상화 & 일반적으로 사용하는 최적화 알고리즘의 Implementation을 제공한다.

In [26]:
import torch

N, D_in, h, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# nn 패키지를 사용해 모델을 sequence of layers로 정의한다.
# nn.Sequential은 다른 모듈을 포함하는 모듈로 그 모듈들을 순차적으로 적용해 출력을 생성
# 각각의 Linear Module은 선형 함수를 사용해 입력으로부터 출력을 계산하고, 내부 Tensor에 가중치와 편향 저장
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out)
)

# 또한 nn 패키지에는 널리 사용하는 손실 함수들에 대한 정의도 포함하고 있음
# 여기에서는 평균 제곱 오차(MSE)를 손실 함수로 사용해보자
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    # forward: 모델에 x를 전달해 예상되는 y값을 계산
    # Module 객체는 __call__ 연산자를 덮어써(overrride) 함수처럼 호출할 수 있게 한다.
    # 이렇게 함으로써 입력 데이터의 Tensor를 모듈에 전달해 출력 데이터의 Tensor를 생성
    y_pred = model(x)
    
    # 손실을 계산하고 출력한다.
    # 예측한 y와 정답인 y를 갖는 Tensor들을 전달하고, 손실 함수는 손실 값을 갖는 Tensor를 반환
    loss = loss_fn(y_pred, y)
    
    if t % 100 == 99:
        print(t, loss.item())
        
    # backpropagation 전 변화도 0으로 만들기
    model.zero_grad()
    
    # backpropagation: 모델의 학습 가능한 모든 매개변수에 대해 gradient를 계산
    # 내부적으로 각 모듈의 매개변수는 requires_grad=True일 때 Tensor 내 저장되므로
    # 이 호출을 통해 모든 모델의 모든 학습 가능한 매개변수의 변화도를 계산하게 된다.
    loss.backward()
    
    # gradient descent를 사용해 가중치를 갱신한다.
    # 각 매개변수는 Tensor이므로 이전에 했던 것과 같이 gradient에  접근할 수 있다.
    optimizer.step()

99 73.01031494140625
199 1.2297476530075073
299 0.0027797359507530928
399 2.0513189156190492e-06
499 1.1729035520602338e-09


## 사용자 정의 nn.Module
- 때때로 기존 모듈의 sequence보다 더 복잡한 모델을 구성해야 할 때가 있다.
- 이 경우 nn.Module의 서브클래스로 새 모듈을 정의하고, 입력 Tensor를 받아 다른 모듈 또는 Tensor의 autograd 연산을 사용해 출력 Tensor를 만드는 forward를 정의

In [33]:
import torch

class TwoLayersNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        - 생성자에서 2개의 nn.Linear 모듈을 생성
        - 멤버 변수로 지정
        """
        super(TwoLayersNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)
        
    def forward(self, x):
        """
        -순전파 함수에서는 입력 데이터의 Tensor를 받고, 출력 데이터의 Tensor 반환해야 함
        - Tensor 상의 임의의 연산자뿐만 아니라 생성자에서 정의한 모듈로 사용할 수 있음
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        
        return y_pred

In [34]:
N, D_in, h, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = TwoLayersNet(D_in, H, D_out)

# 손실 함수와 Optimizer를 만든다.
# SGD 생성자에 model.parameters()를 호출하면 모델의 멤버인
# 2개의 nn.Linear 모듈의 학습 가능한 매개변수들이 포함된다.
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

for t in range(500):
    # forward: 모델에 x를 전달해 pred_y를 계산
    y_pred = model(x)
    
    # 손실을 계산
    loss = criterion(y_pred, y)
    
    if t % 100 == 99:
        print(t, loss.item())
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 2.2947444915771484
199 0.0694611519575119
299 0.004532421939074993
399 0.00035299104638397694
499 2.9988981623318978e-05


## 제어 흐름(Control Flow) + 가중치 공유(Weight Sharing)
- 각 순전파 단계에서 많은 은닉 계층을 갖는 fully-connected ReLU 신경망이 무작위로 0~3 사이의 숫자를 선택하고, 가장 안쪽의 은닉층들을 계산하기 위해 동일한 가중치를 여러번 재사용하는 모델을 구현해보자.

In [35]:
import random
import torch

In [36]:
class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)
        
    def forward(self, x):
        """
        랜덤으로 0 ~ 3 중 하나를 선택하고
        은닉층을 계산하기 위해 여러번 사용한 middle_linear 모듈을 재사용한다.
        각 forward 단계는 동적 연산 그래프를 구성하기 때문에, 모델의 순전파 단계를
        정의할때, 일반적인 파이썬 제어 흐름 연산자 사용 가능하다.
        """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        
        return y_pred
    
N, D_in, h, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = DynamicNet(D_in, H, D_out)

# 손실 함수와 Optimizer를 만든다.
# SGD 생성자에 model.parameters()를 호출하면 모델의 멤버인
# 2개의 nn.Linear 모듈의 학습 가능한 매개변수들이 포함된다.
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)

for t in range(500):
    # forward: 모델에 x를 전달해 pred_y를 계산
    y_pred = model(x)
    
    # 손실을 계산
    loss = criterion(y_pred, y)
    
    if t % 100 == 99:
        print(t, loss.item())
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 20.97184944152832
199 1.5846585035324097
299 0.2157125025987625
399 8.692468643188477
499 0.3674720525741577
